In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

In [2]:
data=pd.read_csv("AVdata.csv")

In [3]:
data.head()

,Unnamed: 0,incident_tehsil,cause_of_emergency,base_to_scene_distance,scene_to_hospital_distance,roadway_feature,road_type,surrounding_area,criticality,incident_time_category,day_of_week,1,2
0,0,Raipur,Multiple Vehicular Incident,13,12,CUR,HIW,FOR,0,5,Wednesday,11,2012
1,1,Raipur,2 Wheeler accidents,1,5,CUR,HIW,AGL,0,5,Friday,11,2011
2,2,Raipur,"Non motorised vehicle accidents(Bullock cart,B...",2,2,INT,LOC,HOSP,0,5,Saturday,7,2009
3,3,Raipur,2 Wheeler accidents,8,4,CUR,HIW,HIL,0,3,Saturday,5,2012
4,4,Sahaspur,2 Wheeler accidents,1,26,CUR,HIW,AGL,0,4,Monday,6,2010


In [4]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [6]:
for var in data:
    data[var]=le.fit_transform(data[var])

In [40]:
data.head()

,incident_tehsil,cause_of_emergency,base_to_scene_distance,scene_to_hospital_distance,roadway_feature,road_type,surrounding_area,incident_time_category,day_of_week,1,2
0,4,9,12,12,0,0,2,5,6,10,3
1,4,0,0,5,0,0,0,5,0,10,2
2,4,10,1,2,1,1,4,5,2,6,0
3,4,0,7,4,0,0,3,3,2,4,3
4,5,0,0,26,0,0,0,4,1,5,1


In [8]:
del data["Unnamed: 0"]

In [9]:
dataset = np.genfromtxt(data, delimiter=",")

In [10]:
dataset[:8]

array([nan, nan, nan, nan, nan, nan, nan, nan])

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
p=data["criticality"]

In [13]:
del data["criticality"]

In [14]:
X=data

In [15]:
X.head()

,incident_tehsil,cause_of_emergency,base_to_scene_distance,scene_to_hospital_distance,roadway_feature,road_type,surrounding_area,incident_time_category,day_of_week,1,2
0,4,9,12,12,0,0,2,5,6,10,3
1,4,0,0,5,0,0,0,5,0,10,2
2,4,10,1,2,1,1,4,5,2,6,0
3,4,0,7,4,0,0,3,3,2,4,3
4,5,0,0,26,0,0,0,4,1,5,1


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,p,test_size=0.1)


In [17]:
model = Sequential()
model.add(Dense(units = 11, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))



In [18]:
class_weight = {0 : 1,
    1: 8,
}

In [19]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size = 32, epochs = 8, class_weight=class_weight)

Epoch 1/8
10599/10599 [==============================] - 1s 52us/step - loss: 0.7574 - acc: 0.9551
Epoch 2/8
10599/10599 [==============================] - 1s 53us/step - loss: 0.6722 - acc: 0.8980
Epoch 3/8
10599/10599 [==============================] - 1s 75us/step - loss: 0.6606 - acc: 0.8633
Epoch 4/8
10599/10599 [==============================] - 1s 50us/step - loss: 0.6570 - acc: 0.8658
Epoch 5/8
10599/10599 [==============================] - 0s 42us/step - loss: 0.6516 - acc: 0.8639
Epoch 6/8
10599/10599 [==============================] - 0s 41us/step - loss: 0.6516 - acc: 0.8605
Epoch 7/8
10599/10599 [==============================] - 1s 54us/step - loss: 0.6501 - acc: 0.8698
Epoch 8/8
10599/10599 [==============================] - 1s 84us/step - loss: 0.6468 - acc: 0.8735


In [20]:
z=model.predict(X_test)

In [21]:
z

array([[0.30591097],
       [0.05670345],
       [0.12925857],
       ...,
       [0.13725525],
       [0.23690155],
       [0.08484364]], dtype=float32)

In [22]:
z=pd.DataFrame(z)

In [23]:
th=[.5,.4,.39,.38,.37,.36,.35,.34,.33,.32,.31,.3,.29,.28,.27,.26,.255,.25,.245,.24,.23,.22,.21,.2,.1,.09,.08,.07,.06,.055,.05,.045,.04,.03,.02,.01,.005]

In [24]:
from sklearn.metrics import roc_auc_score,make_scorer

In [25]:
scores=[]
for k in th:
    z=model.predict(X_test)
    z=pd.DataFrame(z)
    z[z>k]=1
    z[z<=k]=0
    q=roc_auc_score(y_test,z)
    scores.append((q,k))
scores

[(0.5967986840440338, 0.5),
 (0.621707850545001, 0.4),
 (0.6296975831962546, 0.39),
 (0.6283689738074149, 0.38),
 (0.6359158366623886, 0.37),
 (0.6332586178847093, 0.36),
 (0.6292727897181902, 0.35),
 (0.6372625223694438, 0.34),
 (0.6439236456318578, 0.33),
 (0.6678928435856184, 0.32),
 (0.6732253574591927, 0.31),
 (0.6696823990889536, 0.3),
 (0.6856618643914607, 0.29),
 (0.6794616872435423, 0.28),
 (0.6706042913179444, 0.27),
 (0.6644041141700259, 0.26),
 (0.6613040255960665, 0.255),
 (0.655546718244428, 0.25),
 (0.6613221019142821, 0.245),
 (0.6688689647692557, 0.24),
 (0.6719871296614306, 0.23),
 (0.6626868639395528, 0.22),
 (0.6556009471990745, 0.21),
 (0.6613763308689287, 0.2),
 (0.6043726613763309, 0.1),
 (0.5928761229912691, 0.09),
 (0.5804938450136476, 0.08),
 (0.5672258274434664, 0.07),
 (0.5641618915059381, 0.06),
 (0.5641980441423691, 0.055),
 (0.551797689846532, 0.05),
 (0.5402830751432548, 0.045),
 (0.5292113302362575, 0.04),
 (0.5199291408325952, 0.03),
 (0.50885739592559

In [26]:
z[z>0.06]=1
z[z<=0.06]=0

In [28]:
z.head()

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [29]:
for i in z:
    print(i)
    if (i>0.1):
        i=1

0


In [30]:
q=roc_auc_score(y_test,z)

In [31]:
q

0.5004428697962799

In [32]:
test=pd.read_csv('test_TQDFDgg.csv')

In [33]:
del test["incident_location"]
del test["victim_id"]
A=test.incident_time.str.split(':', expand=True)
del A[1]
del A[2]
A.rename(columns={0:'incident_time_category'}, inplace=True)
test1=pd.concat([test,A], axis=1)
itemsB=["00","01","02","03"]
for x in itemsB:
    test1.loc[test1["incident_time_category"]==x,'incident_time_category']="0"
itemsB=["04","05","06","07"]
for x in itemsB:
    test1.loc[test1["incident_time_category"]==x,'incident_time_category']="1"
itemsB=["08","09","10","11"]
for x in itemsB:
    test1.loc[test1["incident_time_category"]==x,'incident_time_category']="2"
itemsB=["12","13","14","15"]
for x in itemsB:
    test1.loc[test1["incident_time_category"]==x,'incident_time_category']="3"
itemsB=["16","17","18","19"]
for x in itemsB:
    test1.loc[test1["incident_time_category"]==x,'incident_time_category']="4"
itemsB=["20","21","22","23"]
for x in itemsB:
    test1.loc[test1["incident_time_category"]==x,'incident_time_category']="5"
test1['incident_date1'] = pd.to_datetime(test1['incident_date'])
test1['day_of_week'] = test1['incident_date1'].dt.weekday_name
B=test1.incident_date.str.split('-', expand=True)
final_test=pd.concat([test1,B] , axis=1)
del final_test["incident_date1"]
del final_test[0]
del final_test["incident_time"]
del final_test["incident_date"]


In [34]:
final_test.head()

,incident_tehsil,cause_of_emergency,base_to_scene_distance,scene_to_hospital_distance,roadway_feature,road_type,surrounding_area,incident_time_category,day_of_week,1,2
0,Sahaspur,MVC - Pedestrian (Run Over/Hit & Run),23,27,INT,LOC,MAR,3,Wednesday,05,2010
1,Doiwala,Multiple Vehicular Incident,5,21,CUR,HIW,AGL,4,Sunday,03,2012
2,Raipur,Multiple Vehicular Incident,3,1,CUR,HIW,AGL,4,Monday,01,2013
3,Sahaspur,Multiple Vehicular Incident,18,14,UNK,LOC,AGL,4,Saturday,05,2014
4,Vikasnagar,2 Wheeler accidents,6,10,INT,LOC,MAR,3,Wednesday,12,2009


In [35]:
categorical_variable=final_test.dtypes[final_test.dtypes=="object"].index.values
for var in categorical_variable:
    final_test[var]=le.fit_transform(final_test[var])

In [36]:
submission=model.predict(final_test)

In [37]:
sub=pd.DataFrame(submission)

In [38]:
sub.head()
sub[sub>0.26]=1
sub[sub<=0.26]=0

In [39]:
sub.to_csv("submission1.csv")